# Layoff Text Classification with OpenAI GPT

In [16]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import openai
OPENAI_API_KEY = 
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

## Pre-process
Prepare the data before sending text to the API. 

In [17]:
# data = pd.read_csv("data/bwp_layoff_merged_full.csv")[["keydevid", "situation_y", "Blue", "White", "Pink", "Unsure"]]
data = pd.read_csv("data/bwp_entry_Yiyang_output.csv", encoding='latin-1')[["keydevid", "situation", "Blue", "White", "Pink", "Unsure"]]
data = data.fillna(0)
data

,keydevid,situation,Blue,White,Pink,Unsure
0,2613181,General Motors Corporation has been prompted b...,1.0,0.0,0.0,0.0
1,182676,EMC Corp. has announced that the company would...,1.0,0.0,0.0,0.0
2,139441653,Efforts to reach a fair labor agreement betwee...,1.0,0.0,0.0,0.0
3,279579155,ConocoPhillips confirmed it will cut up to 230...,0.0,1.0,0.0,0.0
4,4995541,Champion Enterprises Inc. announced that it wi...,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
495,531316273,ADP has filed a notice that it will close its ...,0.0,1.0,0.0,0.0
496,130274896,International Business Machines Corp. announce...,0.0,1.0,0.0,0.0
497,182872,The Xerox Corp. announced that it will layoff ...,1.0,0.0,0.0,0.0
498,6073964,Fidelity National Information Services Inc. wi...,0.0,1.0,0.0,0.0


In [18]:
# Optional
sample_size = 100 # Change this

data_sample = data.sample(n=sample_size, random_state=42)
data_sample

,keydevid,situation,Blue,White,Pink,Unsure
361,242548,Allegheny Technologies has announced that it p...,1.0,0.0,0.0,0.0
73,183323,U.S. Bancorp Piper Jaffray will layoff about 5...,0.0,1.0,0.0,0.0
374,98970926,International Business Machines Corp. has issu...,0.0,0.0,0.0,1.0
155,62476,Kellogg Company announced its plans to lay off...,0.0,0.0,0.0,1.0
104,6202947,News Corp. has flagged job cuts after reportin...,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
347,6339778,Cardinal Health Inc. announced that its Clinic...,1.0,0.0,0.0,0.0
86,224658226,Texas Instruments Inc. has initiated measures ...,1.0,0.0,0.0,0.0
75,1521618,Ultimate Electronics Inc. will close its consu...,0.0,1.0,1.0,0.0
438,129969,Adobe Systems Incorporated. said it is dismiss...,0.0,1.0,0.0,0.0


## Fine-tuning Preparation

In [19]:
# prompt_text = "Decide whether the following layoff text primarily concerns white-collar, blue-collar, pink-collar workers, or unsure. "
prompt_text = "Decide whether the following layoff text primarily concerns white-collar, blue-collar, or pink-collar workers (shop assistants, retail clerks are pink-collar). "

In [20]:
def generate_full_prompt(layoff_text):
    return f"{prompt_text} \n\nText: \"{layoff_text}\"\nWorker type:"

In [21]:
# fine_tune_sample_size = 25
# data_blue = data[data["Blue"] == 1].sample(n=fine_tune_sample_size, random_state=42)
# data_blue["completion"] = ["Blue"] * fine_tune_sample_size
# data_white = data[data["White"] == 1].sample(n=fine_tune_sample_size, random_state=42)
# data_white["completion"] = ["White"] * fine_tune_sample_size
# data_pink = data[data["Pink"] == 1].sample(n=fine_tune_sample_size, random_state=42)
# data_pink["completion"] = ["Pink"] * fine_tune_sample_size
# data_unsure = data[data["Unsure"] == 1].sample(n=fine_tune_sample_size, random_state=42)
# data_unsure["completion"] = ["Unsure"] * fine_tune_sample_size
# fine_tune_data = pd.concat([data_blue, data_white, data_pink, data_unsure], axis=0).sample(frac=1, random_state=42)
# fine_tune_data = fine_tune_data[["situation_y", "completion"]].rename(columns={"situation_y": "prompt"})
# fine_tune_data["prompt"] = fine_tune_data["prompt"].apply(generate_full_prompt)
# fine_tune_data

In [22]:
# fine_tune_data.to_json("data/layoff.jsonl", orient='records', lines=True)

In [ ]:
# !openai tools fine_tunes.prepare_data -f data/layoff.jsonl -q

In [ ]:
# !openai api fine_tunes.create -t "data/layoff_prepared_train.jsonl" -v "data/layoff_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 4 -m ada

In [8]:
!openai api fine_tunes.follow -i ft-yJ9dv1jpFpSyYynfClngXk8W

[2023-03-22 18:55:52] Created fine-tune: ft-yJ9dv1jpFpSyYynfClngXk8W
[2023-03-22 18:59:06] Fine-tune costs $0.01
[2023-03-22 18:59:07] Fine-tune enqueued. Queue number: 0
[2023-03-22 18:59:08] Fine-tune started
[2023-03-22 18:59:35] Completed epoch 1/4
[2023-03-22 18:59:49] Completed epoch 2/4
[2023-03-22 19:00:02] Completed epoch 3/4
[2023-03-22 19:00:14] Completed epoch 4/4
[2023-03-22 19:00:40] Uploaded model: ada:ft-personal-2023-03-23-02-00-40
[2023-03-22 19:00:41] Uploaded result file: file-P0shtnP20U22wcEkYo9W7kfp
[2023-03-22 19:00:41] Fine-tune succeeded


Traceback (most recent call last):
  File "C:\Users\Yiyan\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Yiyan\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Yiyan\AppData\Local\Programs\Python\Python38\Scripts\openai.exe\__main__.py", line 7, in <module>
  File "C:\Users\Yiyan\AppData\Local\Programs\Python\Python38\lib\site-packages\openai\_openai_scripts.py", line 63, in main
    args.func(args)
  File "C:\Users\Yiyan\AppData\Local\Programs\Python\Python38\lib\site-packages\openai\cli.py", line 515, in follow
    cls._stream_events(args.id)
  File "C:\Users\Yiyan\AppData\Local\Programs\Python\Python38\lib\site-packages\openai\cli.py", line 558, in _stream_events
    sys.stdout.write("\nJob complete! Status: succeeded \U0001f389")
UnicodeEncodeError: 'gbk' codec can't encode character '\U0001f389' in position 

In [18]:
# ft_model = 'ada:ft-personal-2023-03-23-02-00-40'

## Get Classification Result

In [23]:
def get_classification_result(layoff_text):
    try: 
        response = openai.Completion.create(
          model="text-davinci-003", #"text-davinci-003", ft_model
          prompt=f"{prompt_text} \n\nText: \"{layoff_text}\"\nWorker type:",
          temperature=0,
          max_tokens=20,
          top_p=1.0,
          frequency_penalty=0.5,
          presence_penalty=0.0
        )
        return response["choices"][0]["text"]
    except:
        print("Some error occurred.")
        return None

In [24]:
# this code will run all the API query
# run this cell with caution!

# Comment out the following line if you intend to run this cell
# assert 0 == 1, "API fetch aborted. "

gpt_class = []
for i in tqdm(range(len(data_sample))):
    gpt_class.append(get_classification_result(data_sample.iloc[i, 1]))
gpt_class = pd.Series(gpt_class)

  0%|          | 0/100 [00:00<?, ?it/s]

## Post-process
Get accuracy by comparing to human classification.

In [25]:
def get_accuracy(r):
    if r["gpt_class"] == "blue":
        return r["Blue"] == 1
    elif r["gpt_class"] == "white":
        return r["White"] == 1
    elif r["gpt_class"] == "pink":
        return r["Pink"] == 1
    elif r["gpt_class"] == "unsure":
        return r["Unsure"] == 1
    else:
        return None

In [26]:
data_sample = data_sample.reset_index(drop=True)
data_sample["gpt_class"] = gpt_class.str.lower().str.replace(r"-collar", "").str.replace(" ", "")
data_sample

,keydevid,situation,Blue,White,Pink,Unsure,gpt_class
0,242548,Allegheny Technologies has announced that it p...,1.0,0.0,0.0,0.0,white
1,183323,U.S. Bancorp Piper Jaffray will layoff about 5...,0.0,1.0,0.0,0.0,white
2,98970926,International Business Machines Corp. has issu...,0.0,0.0,0.0,1.0,white
3,62476,Kellogg Company announced its plans to lay off...,0.0,0.0,0.0,1.0,blue
4,6202947,News Corp. has flagged job cuts after reportin...,0.0,0.0,0.0,1.0,white
...,...,...,...,...,...,...,...
95,6339778,Cardinal Health Inc. announced that its Clinic...,1.0,0.0,0.0,0.0,white
96,224658226,Texas Instruments Inc. has initiated measures ...,1.0,0.0,0.0,0.0,white
97,1521618,Ultimate Electronics Inc. will close its consu...,0.0,1.0,1.0,0.0,pink
98,129969,Adobe Systems Incorporated. said it is dismiss...,0.0,1.0,0.0,0.0,white


In [27]:
def post_process(data_sample):
    data_sample = data_sample.reset_index()
    data_sample["gpt_class"] = gpt_class.str.lower().str.replace(r"-collar", "").str.replace(" ", "")
    data_sample_accuracy = data_sample.dropna()
    accuracy = data_sample_accuracy.apply(get_accuracy, axis=1)
    data_sample_accuracy["match"] = accuracy
    print(np.mean(accuracy))
    return data_sample_accuracy

In [28]:
data_sample_accuracy = post_process(data_sample)
data_sample_accuracy

0.6767676767676768


,index,keydevid,situation,Blue,White,Pink,Unsure,gpt_class,match
0,0,242548,Allegheny Technologies has announced that it p...,1.0,0.0,0.0,0.0,white,False
1,1,183323,U.S. Bancorp Piper Jaffray will layoff about 5...,0.0,1.0,0.0,0.0,white,True
2,2,98970926,International Business Machines Corp. has issu...,0.0,0.0,0.0,1.0,white,False
3,3,62476,Kellogg Company announced its plans to lay off...,0.0,0.0,0.0,1.0,blue,False
4,4,6202947,News Corp. has flagged job cuts after reportin...,0.0,0.0,0.0,1.0,white,False
...,...,...,...,...,...,...,...,...,...
95,95,6339778,Cardinal Health Inc. announced that its Clinic...,1.0,0.0,0.0,0.0,white,False
96,96,224658226,Texas Instruments Inc. has initiated measures ...,1.0,0.0,0.0,0.0,white,False
97,97,1521618,Ultimate Electronics Inc. will close its consu...,0.0,1.0,1.0,0.0,pink,True
98,98,129969,Adobe Systems Incorporated. said it is dismiss...,0.0,1.0,0.0,0.0,white,True


In [29]:
data_sample_accuracy[data_sample_accuracy["match"]==False].to_csv("mismatch_examples_without_unsure_s42_large_pink.csv", index=False)

In [55]:
data_sample_accuracy.loc[15, "situation_y"]

"Cisco has confirmed 225 of its 6,000 reported global layoffs hit its Research Triangle Park campus. The number represents nearly 4% of company's 5,500 headcount in RTP. The company is in the process of expanding and adding 550 jobs to RTP - a plan that continues, in spite of layoffs."